In [245]:
import hashlib
import time
import ecdsa
import random
import string

### Defines a wallet with a public and private key and balance

In [246]:
class Wallet:
    def __init__(self, name):
        # Generate a random private and public key pair for the wallet using secp256k1 curve
        self.name = name
        self.private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1).to_string().hex()
        self.public_key = hashlib.sha256(self.private_key.encode()).hexdigest()
        self.balance = 0

    def add_balance(self, amount):
        self.balance += amount
        return self
    
    def display_all_wallet(self):
        print(f"Wallet {self.name}:")
        print(f"  Public Key: {self.public_key}")
        print(f"  Private Key: {self.private_key}")
        print(f"  Balance: {self.balance} units")

    def display_public_key(self):
        print(f"Public Key: {self.public_key}")

    def display_private_key(self):
        print(f"Private Key: {self.private_key}")

### Defines the transaction sender, recipient, amount, time, id

In [247]:
class Transaction:
    def __init__(self, sender, recipient, amount):
        self.sender = sender
        self.recipient = recipient
        self.amount = amount
        self.timestamp = time.time()
        self.transaction_id = self.calculate_transaction_id()

    def calculate_transaction_id(self):
        # Hash the transaction details to create a unique identifier
        data = f"{self.sender.public_key if self.sender else 'None'}{self.recipient.public_key}{self.amount}{self.timestamp}"
        return hashlib.sha256(data.encode()).hexdigest()

    def execute_transaction(self):
        if self.sender and self.sender.balance < self.amount:
            raise ValueError("Insufficient funds for the transaction.")

        # Deduct the amount from the sender's balance (if sender is not None)
        if self.sender:
            self.sender.balance -= self.amount

        # Add the amount to the recipient's balance
        self.recipient.balance += self.amount


    def display_transaction(self):
        formatted_timestamp = time.strftime("%m/%d/%Y %H:%M:%S", time.localtime(self.timestamp))
        print("Transaction:")
        print(f"  Sender Public Key: {self.sender.public_key if self.sender else 'None'}")
        print(f"  Recipient Public Key: {self.recipient.public_key}")
        print(f"  Amount: {self.amount}")
        print(f"  Timestamp: {formatted_timestamp}")
        print(f"  Transaction ID: {self.transaction_id}")

### Defines the Block which has a number, timestamp, transaction data, previous hash and its hash

In [248]:
class Block:
    def __init__(self, block_number, previous_hash, transactions):
        self.block_number = block_number
        self.timestamp = time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        data = (
            str(self.block_number)
            + str(self.timestamp)
            + str([transaction.transaction_id for transaction in self.transactions])
            + str(self.previous_hash)
        )
        return hashlib.sha256(data.encode()).hexdigest()

### Defines the blockchain Creates the first block when called and then has a function to add_blocks (in a real blockchain the Blockchain.chain would be immutable but in this case it isn't)

In [266]:
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.current_transactions = []

    def create_genesis_block(self):
        # Create the first block in the chain (Genesis Block)
        return Block(0, "0", [])

    def mine_block(self, miner_wallet, miner_reward):
        # Create a new block with the pending transactions
        new_block = Block(len(self.chain), self.chain[-1].hash, self.current_transactions)
        # Reset the list of pending transactions
        self.current_transactions = []

        # Reward the miner for mining a new block
        miner_transaction = Transaction(None, miner_wallet, miner_reward)
        miner_transaction.execute_transaction()

        # Execute all transactions in the mined block
        for transaction in new_block.transactions:
            transaction.execute_transaction()

        # Add the new block to the blockchain
        self.chain.append(new_block)

    def add_transaction(self, transaction):
        # Ensure that the transaction is valid before adding it to the blockchain
        if not isinstance(transaction, Transaction):
            raise ValueError("Invalid transaction. Must be an instance of the Transaction class.")

        if transaction.sender:
            if transaction.sender.balance < transaction.amount:
                raise ValueError("Insufficient funds for the transaction.")

            # Deduct the amount from the sender's balance
            transaction.sender.balance -= transaction.amount

        # Add the transaction to the list of current transactions
        self.current_transactions.append(transaction)

    def display_chain(self):
        for block in self.chain:
            formatted_timestamp = time.strftime("%m/%d/%Y %H:%M:%S", time.localtime(block.timestamp))
            transaction_ids = [transaction.transaction_id for transaction in block.transactions]
            print(f"Block #{block.block_number} | Hash: {block.hash}")
            print(f"Timestamp: {formatted_timestamp}")
            print(f"Transaction IDs: {transaction_ids}")
            print("--------------")

### Mining

--------------
1. Transaction: Transfer 10 BTC from Alice to Bob
2. Transaction: Transfer 5 BTC from Bob to Charlie
3. Transaction: Transfer 8 BTC from Charlie to Alice
--------------

In [267]:
if __name__ == "__main__":
    # Create wallets
    alice_wallet = Wallet("Alice")
    bob_wallet = Wallet("Bob")
    charlie_wallet = Wallet("Charlie")
    miner_wallet = Wallet("Miner")

    # Initial balances
    alice_wallet.add_balance(100)
    bob_wallet.add_balance(50)
    charlie_wallet.add_balance(30)

# Display initial balances
alice_wallet.display_all_wallet()
bob_wallet.display_all_wallet()
charlie_wallet.display_all_wallet()

Wallet Alice:
  Public Key: 40e64ab6a31bc94ea4c53c5ed0ce2758debb6f6c31b5f2efe77a272481bf79fb
  Private Key: fca7c2d361a0f8bbf374ee50ff2196e30a473c9448e08263d02b8fd01bff8bb8
  Balance: 100 units
Wallet Bob:
  Public Key: 871e92734d9e9375821ce7b549d8b9a36f40f369f08f5e50d48eaf6aab7a6126
  Private Key: 2c6ac928db402e5de10871747a221ebdca67d75c8c1e92d6cd00cf425e443e03
  Balance: 50 units
Wallet Charlie:
  Public Key: 1bba165285537222c9b9a30740a0b494d8e9cb00485645214a329d2753712baa
  Private Key: 63c4d5eff190fa87da9e3b48a572bccb357427f3a61c756e8999c5f37bc2b028
  Balance: 30 units


In [268]:
# Create a blockchain
my_blockchain = Blockchain()

In [269]:
my_blockchain.display_chain() # Should only display the genesis block

Block #0 | Hash: 48d1690ec1ad83be939df9bbf9813ed0ec53d771c21a9a956f971358bd5c5e67
Timestamp: 12/05/2023 02:23:11
Transaction IDs: []
--------------


### Cria as transações e as executa (ainda não terminaram)

In [270]:
# Create transactions
transaction1 = Transaction(alice_wallet, bob_wallet, 10)
transaction2 = Transaction(bob_wallet, charlie_wallet, 5)
transaction3 = Transaction(charlie_wallet, alice_wallet, 8)

# Add transactions to the blockchain
my_blockchain.add_transaction(transaction1)
my_blockchain.add_transaction(transaction2)
my_blockchain.add_transaction(transaction3)

### Balanço Inicial das 3 Carteiras

In [271]:
alice_wallet.display_all_wallet()
bob_wallet.display_all_wallet()
charlie_wallet.display_all_wallet()

Wallet Alice:
  Public Key: 40e64ab6a31bc94ea4c53c5ed0ce2758debb6f6c31b5f2efe77a272481bf79fb
  Private Key: fca7c2d361a0f8bbf374ee50ff2196e30a473c9448e08263d02b8fd01bff8bb8
  Balance: 90 units
Wallet Bob:
  Public Key: 871e92734d9e9375821ce7b549d8b9a36f40f369f08f5e50d48eaf6aab7a6126
  Private Key: 2c6ac928db402e5de10871747a221ebdca67d75c8c1e92d6cd00cf425e443e03
  Balance: 45 units
Wallet Charlie:
  Public Key: 1bba165285537222c9b9a30740a0b494d8e9cb00485645214a329d2753712baa
  Private Key: 63c4d5eff190fa87da9e3b48a572bccb357427f3a61c756e8999c5f37bc2b028
  Balance: 22 units


### Publico Todos podem ver

In [272]:
transaction1.display_transaction()
transaction2.display_transaction()
transaction3.display_transaction()

Transaction:
  Sender Public Key: 40e64ab6a31bc94ea4c53c5ed0ce2758debb6f6c31b5f2efe77a272481bf79fb
  Recipient Public Key: 871e92734d9e9375821ce7b549d8b9a36f40f369f08f5e50d48eaf6aab7a6126
  Amount: 10
  Timestamp: 12/05/2023 02:23:15
  Transaction ID: 4afec8eba43c268ba38eda991cda2bc1643f07dd06dd4cb35a44e810e24cb066
Transaction:
  Sender Public Key: 871e92734d9e9375821ce7b549d8b9a36f40f369f08f5e50d48eaf6aab7a6126
  Recipient Public Key: 1bba165285537222c9b9a30740a0b494d8e9cb00485645214a329d2753712baa
  Amount: 5
  Timestamp: 12/05/2023 02:23:15
  Transaction ID: 816123fe45c7bc195742da2079c998f7d623e1f88cb67f9a5c25930229478c14
Transaction:
  Sender Public Key: 1bba165285537222c9b9a30740a0b494d8e9cb00485645214a329d2753712baa
  Recipient Public Key: 40e64ab6a31bc94ea4c53c5ed0ce2758debb6f6c31b5f2efe77a272481bf79fb
  Amount: 8
  Timestamp: 12/05/2023 02:23:15
  Transaction ID: fe6376ba38f536dcf1013f2079b1b1cb4afa9a988e8cc37fcf5d6cc93a9980fa


### Bloco Minerado e final da transação

In [273]:
# Mine a block with a reward for the miner
miner_reward = 5
my_blockchain.mine_block(miner_wallet, miner_reward)

# Display the blockchain
my_blockchain.display_chain()

Block #0 | Hash: 48d1690ec1ad83be939df9bbf9813ed0ec53d771c21a9a956f971358bd5c5e67
Timestamp: 12/05/2023 02:23:11
Transaction IDs: []
--------------
Block #1 | Hash: 37033edb85105bdb857b314a6b689ba462e75aad7cf9df83c6e7303531b2e560
Timestamp: 12/05/2023 02:23:30
Transaction IDs: ['4afec8eba43c268ba38eda991cda2bc1643f07dd06dd4cb35a44e810e24cb066', '816123fe45c7bc195742da2079c998f7d623e1f88cb67f9a5c25930229478c14', 'fe6376ba38f536dcf1013f2079b1b1cb4afa9a988e8cc37fcf5d6cc93a9980fa']
--------------


### Recomepensa do miner

In [258]:
miner_wallet.display_all_wallet()

Wallet Miner:
  Public Key: 310cdb87a48f4494589e0e26c2490825fbeaeb587618630e8f776b61092c2205
  Private Key: e9ee0ab2f804036316cf95379799703faad961af923fbcae0c3e264b08b10522
  Balance: 5 units


### Resultado após Finalização das Transações

In [274]:
alice_wallet.display_all_wallet()
bob_wallet.display_all_wallet()
charlie_wallet.display_all_wallet()

Wallet Alice:
  Public Key: 40e64ab6a31bc94ea4c53c5ed0ce2758debb6f6c31b5f2efe77a272481bf79fb
  Private Key: fca7c2d361a0f8bbf374ee50ff2196e30a473c9448e08263d02b8fd01bff8bb8
  Balance: 88 units
Wallet Bob:
  Public Key: 871e92734d9e9375821ce7b549d8b9a36f40f369f08f5e50d48eaf6aab7a6126
  Private Key: 2c6ac928db402e5de10871747a221ebdca67d75c8c1e92d6cd00cf425e443e03
  Balance: 50 units
Wallet Charlie:
  Public Key: 1bba165285537222c9b9a30740a0b494d8e9cb00485645214a329d2753712baa
  Private Key: 63c4d5eff190fa87da9e3b48a572bccb357427f3a61c756e8999c5f37bc2b028
  Balance: 19 units
